In [1]:
import numpy as np
import pickle
import sys
sys.path.append("..")
from dataAggregation.json_processor import json_unzip
from utils.preprocess import preprocess


[nltk_data] Downloading package stopwords to C:\Users\Johnson
[nltk_data]     Lui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Johnson
[nltk_data]     Lui\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Johnson
[nltk_data]     Lui\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load Compressed Dataset
zipped_dataset = pickle.load(open("../../data/movies_review_json_zip.p", "rb"))

In [3]:
# Unzip the Dataset
movie_data = json_unzip(zipped_dataset)

In [4]:
# Configuration
topK_movie = 5

In [5]:
# Create Movie ID
movie_id_list = list(movie_data)


In [6]:
# Building Vocabulary
preprocessed_corpus = []
for m_id in movie_id_list:
    # Put Movie Title and Plot as the summary of the movie
    movie_name = movie_data[m_id]["localized_title"]
    summary = movie_data[m_id]["summary"].split("\nPlot: ")
    summary = summary[1] if len(summary) > 1 else ""
        
    preprocessed_corpus.append(
        preprocess(movie_name, is_sw_remove=False) + preprocess(summary)
    )


##### Evaluation

In [8]:
from utils.evaluator import MetricsEvaluator

In [9]:
# Create Metrics Evaluator
evaluator = MetricsEvaluator(movie_data)

##### Random walk

In [6]:
from similarity.random import RandomCalculator

random_calculator = RandomCalculator(movie_data, topK_movie)
similar_movies_result_random = random_calculator.generateSimilarMovies()


In [10]:
np.mean(evaluator.evaluate(similar_movies_result_random))


1.6449281091741155

##### Jaccard Similarity

In [9]:
from similarity.jaccard import JaccardCalculator

# Create Jaccard Similarity Calculator
jaccard_calculator = JaccardCalculator(preprocessed_corpus)
# Build Simiarity Matrix
jaccard_calculator.buildJaccardSimilarityMatrix()


In [10]:
# Calculate Similar Movie Result
similar_movies_result = {}
for i in range(len(preprocessed_corpus)):
    similar_movies = (-jaccard_calculator.sim_matrix[i]).argsort()[:topK_movie + 1]
    similar_movies = [movie_id_list[j] for j in similar_movies if j != i][:topK_movie]

    assert movie_id_list[i] not in similar_movies, "Movie ID in Similar Movies List"

    similar_movies_result[movie_id_list[i]] = similar_movies



In [13]:
np.mean(evaluator.evaluate(similar_movies_result))

2.338252519004315